#### Import dependencies

In [1]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Importing functions from the modules in the qseg package
# from qseg.graph_utils import image_to_grid_graph, draw, draw_graph_cut_edges
# from qseg.dwave_utils import dwave_solver, annealer_solver
# from qseg.utils import decode_binary_string

# Additional necessary imports
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from qiskit_optimization.applications import Maxcut
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [3]:
import gurobipy as gp
from gurobipy import GRB

In [4]:
import utils
import dwave_utils

In [5]:
import os
import time
import csv

In [6]:
import itertools

In [7]:
import random as rnd

#### Generate Problem Instance

In [8]:
# def generate_induced_subgraph_game(distribution, n_agents, sparsity=0.0, **kwargs):
#     induced_subgraph_game = {}
#     keys = list(itertools.combinations(range(1,n_agents+1), 2))
#     totalinteractions = len(keys)
#     values = distribution(totalinteractions, **kwargs)
#     for i,key in enumerate(keys):
#         induced_subgraph_game[','.join(map(str,key))] = round(values[i],2)
#     return induced_subgraph_game

In [9]:
def generate_induced_subgraph_game(distribution, n_agents, sparsity=0.0, seed = 123, **kwargs):
    """
    generate_induced_subgraph_game: To generate a . 
    :distribution: distribution function name to sample the edge weights from. (dtype: function) 
    :n_agents: number of agents (dtype: int) 
    :sparsity: describes the connectivity of the graph edges. \
    If 0.0, generates a fully connected graph. \
    If 1.0 generates a connected graph with minimal edges. 
    (dtype: float, range: [0,1]) 
    :returns: induced subgraph game with agents pairs as key and their synergy as float value. 
    (dtype: dictionary) 
    """
    induced_subgraph_game = {}
    #### D: tyring to set a seed to get compareable results
    connected_edges = [(i + 1, j + 1) for i, j in nx.random_tree(n_agents, seed=seed).edges] 
    extra_edges = list(set(itertools.combinations(range(1, n_agents + 1), 2)).difference(connected_edges))
    #### D: seed
    rnd.seed(seed)
    sampled_edges = rnd.sample(extra_edges, int((1 - sparsity) * len(extra_edges)))
    keys = sorted(connected_edges + sampled_edges)
    totalinteractions = len(keys)
    np.random.seed(seed=seed)
    values = distribution(totalinteractions, **kwargs)
    for i, key in enumerate(keys):
        induced_subgraph_game[','.join(map(str, key))] = round(values[i], 2)
    G=nx.Graph()
    G.add_nodes_from(np.arange(0,n_agents,1))
    elist = [tuple((int(x)-1 for x in key.split(',')))+tuple([induced_subgraph_game[key]*-1]) for key in induced_subgraph_game]
    G.add_weighted_edges_from(elist)
    return induced_subgraph_game, G

In [10]:
# def get_nx_e_list(h, w):
#     nx_elist = []
#     for i in range(h*w):
#         x, y = i//w, i%w
#         if x > 0:
#             nx_elist.append(((x,y),(x-1,y)))
#         if y > 0:
#             nx_elist.append(((x,y),(x,y-1)))
#     return nx_elist

In [11]:
# def generate_problem_instance(height,width):
#     nx_elist = get_nx_e_list(height,width)
#     G = nx.grid_2d_graph(height,width)
#     edge_weights = np.random.uniform(low=-1, high=1, size=len(nx_elist))
#     G.add_weighted_edges_from([(node[0],node[1], w) for node,w in zip(nx_elist, edge_weights)])
#     return G

#### D-Wave QUBO Solver

In [12]:
def dwave_solver(linear, quadratic, private_token, runs=10000, **kwargs):
  """
  Solve a binary quadratic model using D-Wave sampler.

  Parameters:
  linear (dict): Linear coefficients of the model.
  quadratic (dict): Quadratic coefficients of the model.
  private_token (str): API token for D-Wave.
  runs (int): Number of reads for the sampler.

  Returns:
  dimod.SampleSet: Sample set returned by D-Wave sampler.
  float: Connection time.
  float: Embedding time.
  float: Response time.
  """
  vartype = dimod.BINARY
  bqm = dimod.BinaryQuadraticModel(linear, quadratic, 0.0, vartype)
  start_time = time.time()
  dwave_sampler = DWaveSampler(token = private_token, solver={'topology__type': 'pegasus'})
  connection_time = time.time() - start_time

  start_time = time.time()
  sampler = EmbeddingComposite(dwave_sampler)
  embedding_time = time.time() - start_time
  start_time = time.time()
  sample_set = sampler.sample(bqm, num_reads=runs)
  response_time = time.time() - start_time
  return sample_set, connection_time, embedding_time, response_time

def annealer_solver(G, private_token, n_samples=2000, **kwargs):
  """
  Solve the Maxcut problem on graph G using a D-Wave annealer.

  Parameters:
  G (networkx.Graph): Graph for which Maxcut is to be solved.
  private_token (str): API token for D-Wave.
  n_samples (int): Number of samples to collect.

  Returns:
  pandas.DataFrame: Dataframe containing samples.
  dict: Dictionary containing information about execution times.
  """
  start_time = time.time()
  w = -1 * nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)
  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}
  problem_formulation_time = time.time() - start_time
  sample_set, connection_time, embedding_time, response_time = dwave_solver(linear, quadratic, private_token, runs=n_samples)
  info_dict = sample_set.info['timing'].copy()

  start_time = time.time()
  samples_df = sample_set.to_pandas_dataframe()
  sample_fetch_time = time.time() - start_time

  info_dict['problem_formulation_time'] = problem_formulation_time
  info_dict['connection_time'] = connection_time
  info_dict['embedding_time'] = embedding_time
  info_dict['response_time'] = response_time
  info_dict['sample_fetch_time'] = sample_fetch_time
  return samples_df, info_dict

#### Gurobi QUBO solver

In [13]:
def gurobi_qubo_solver(G):
  w = -1 * nx.adjacency_matrix(G).todense()
  max_cut = Maxcut(w)
  qp = max_cut.to_quadratic_program()
  linear = qp.objective.linear.coefficients.toarray(order=None, out=None)
  quadratic = qp.objective.quadratic.coefficients.toarray(order=None, out=None)

  linear = {int(idx):-round(value,2) for idx,value in enumerate(linear[0])}
  quadratic = {(int(iy),int(ix)):-quadratic[iy, ix] for iy, ix in np.ndindex(quadratic.shape) if iy<ix and abs(quadratic[iy, ix])!=0}

  qubo_matrix = np.zeros([len(linear),len(linear)])
  for key,value in linear.items():
    qubo_matrix[int(key),int(key)] = value
  for key,value in quadratic.items():
    qubo_matrix[int(key[0]),int(key[1])] = value/2
    qubo_matrix[int(key[1]),int(key[0])] = value/2

  n = qubo_matrix.shape[0]
  model = gp.Model()
  x = model.addVars(n, vtype=GRB.BINARY)
  obj_expr = gp.quicksum(qubo_matrix[i, j] * x[i] * x[j] for i in range(n) for j in range(n))
  model.setObjective(obj_expr)
  model.setParam('OutputFlag', 0)
  model.optimize()

  if model.status == GRB.OPTIMAL:
    solution = [int(x[i].X) for i in range(n)]
    binary_string = ''.join(str(bit) for bit in solution)
    return binary_string, model.objVal
  else:
    return None, None

#### Experimental settings

In [14]:
# seeds = [111,222,333,444,555]

seed = 222

dwave_annealer_solver = False#@param {type:"boolean"}
gurobipy_qubo_solver = True#@param {type:"boolean"}

solver_flags = ''.join(map(str,map(int,[dwave_annealer_solver, gurobipy_qubo_solver])))


table_contents = []

# heights = np.arange(2, 51).tolist()
report_filename = 'optimalSplit_report_' + solver_flags + '_' +  str(seed) + '.txt'
report_filename_csv = 'optimalSplit_report_' + solver_flags + '_' +  str(seed) + '.csv'
qputimes_filename = 'optimalSplit_qputimes_' + solver_flags + '_' +  str(seed) + '.csv'

qpu_output_path = os.path.join('singleSplitresults', f'{seed}')

n_samples = 2000

In [15]:
# Create the directory if it doesn't exist
# os.makedirs('optimalSplitResults', exist_ok=True)

fieldnames = ['n', 'sparsity', 'distribution', 'seed', 'solution_gurobi', 'value_gurobi', 'gurobi_tte']
if not os.path.exists(report_filename_csv):
  with open(report_filename_csv, mode='w', newline='') as report_file:
        writer = csv.DictWriter(report_file, fieldnames=fieldnames)
        # Write the header row
        writer.writeheader()

In [16]:
# # Create the directory if it doesn't exist
# os.makedirs(qpu_output_path, exist_ok=True)

# fieldnames = ['qpu_sampling_time', 'qpu_anneal_time_per_sample', 'qpu_readout_time_per_sample', 'qpu_access_time', 'qpu_access_overhead_time', 'qpu_programming_time', 'qpu_delay_time_per_sample', 'post_processing_overhead_time', 'total_post_processing_time', 'problem_formulation_time', 'connection_time', 'embedding_time', 'response_time', 'sample_fetch_time', 'n', 'sparsity']
# if dwave_annealer_solver and not os.path.exists(qputimes_filename):
#   with open(qputimes_filename, mode='w', newline='') as qputime_file:
#         writer = csv.DictWriter(qputime_file, fieldnames=fieldnames)
#         # Write the header row
#         writer.writeheader()

#### Executions

In [17]:
distributions = [
    # utils.laplace,
    # utils.normal,
    utils.random
]

# sparsities = [0.0,0.5,1.0]
sparsities = [0.5, 1.0]

seeds = [333]
# seeds = [111]

n_agents = np.arange(474,521)

In [18]:
for seed in seeds:
  for distribution in distributions:
    for n in n_agents:
      for sparsity in sparsities:
        if n > 80 and sparsity == 0.5:
          continue
        base_path = os.path.join(qpu_output_path, str(seed),f"sparsity{sparsity}")
        os.makedirs(base_path, exist_ok=True)
        print(f'seed: {seed}, number of agents: {n}')
        np.random.seed(seed=seed)
        isg, G = generate_induced_subgraph_game(distribution, n, sparsity=sparsity, seed = seed)
        start_time = time.time()
        if dwave_annealer_solver:
          sample_set_df, info_dict = annealer_solver(G, private_token='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx', n_samples = n_samples)
          dwave_annealer_tte = (time.time() - start_time)
          print('dwave_annealer_tte',dwave_annealer_tte)
          dwave_annealer_solution_first = ''.join(sample_set_df.iloc[0][:-3].astype(int).astype(str))
          dwave_annealer_solution_most = ''.join(sample_set_df.sort_values(by=['num_occurrences', 'energy'], ascending=[False, True]).iloc[0][:-3].astype(int).astype(str))

          dwave_annealer_value_first = sample_set_df.iloc[0][-2:-1][0]
          dwave_annealer_value_most = sample_set_df.sort_values(by=['num_occurrences', 'energy'], ascending=[False, True]).iloc[0][-2:-1][0]

          print('dwave_annealer_value_lowest_energy',dwave_annealer_value_first)
          print('dwave_annealer_value_most_occurred',dwave_annealer_value_most)

          print("ANNEALER DONE!")
          qputime_dic = info_dict.copy()
          qputime_dic['n'] = n
          qputime_dic['sparsity'] = sparsity
          # qputime_dic['Width'] = width
          print('qputime_dic',qputime_dic)
  
          sample_set_df.to_csv(os.path.join(base_path, f'{n}.csv'), index=False)

          with open(qputimes_filename, mode='a', newline='') as qpu_times_file:
            writer = csv.DictWriter(qpu_times_file, fieldnames=qputime_dic.keys())
            writer.writerow(qputime_dic)
        else:
          dwave_annealer_solution, dwave_annealer_value = None, None
  
  
        start_time = time.time()
        if gurobipy_qubo_solver:
          try:
            gurobi_qubo_solution, gurobi_qubo_value = gurobi_qubo_solver(G)
            gurobi_qubo_tte = (time.time() - start_time)
            print('gurobi_qubo_value',gurobi_qubo_value)
            print('gurobi_qubo_tte:',gurobi_qubo_tte)
            print("GUROBI QUBO DONE!")
          except Exception as e:
            print("Gurobi QUBO execution is unsuccessful due to "+ str(e))
            gurobi_qubo_solution, gurobi_qubo_value = None, None
        else:
          gurobi_qubo_solution, gurobi_qubo_value = None, None

        row = []
        row_d = {}

        row.append(int(n))
        row.append(np.round(sparsity,4))
        row.append(distribution.__name__)
        row.append(seed)


        row_d['n'] = int(n)
        row_d['sparsity'] = np.round(sparsity,4)
        row_d['distribution'] = distribution.__name__
        row_d['seed'] = seed
        # row.append(int(width))

        if dwave_annealer_solver:
          row.append(''.join(map(str, map(int, (dwave_annealer_solution_first)))))
          row.append(np.round(dwave_annealer_value_first,4))
          row.append(''.join(map(str, map(int, (dwave_annealer_solution_most)))))
          row.append(np.round(dwave_annealer_value_most,4))
          row.append(np.round(dwave_annealer_tte,4))

          row_d['solution_first'] = ''.join(map(str, map(int, (dwave_annealer_solution_first))))
          row_d['value_first'] = np.round(dwave_annealer_value_first,4)
          row_d['solution_most'] = ''.join(map(str, map(int, (dwave_annealer_solution_most))))
          row_d['value_most'] = np.round(dwave_annealer_value_most,4)
          row_d['annealer_tte'] = np.round(dwave_annealer_tte,4)

        if gurobipy_qubo_solver:
          if gurobi_qubo_solution:
            row.append(''.join(map(str, map(int, (gurobi_qubo_solution)))))
            row_d['solution_gurobi'] = ''.join(map(str, map(int, (gurobi_qubo_solution))))
          else:
            row.append(''.join(['0']*G.number_of_nodes()))
            row_d['solution_gurobi'] = ''.join(['0']*G.number_of_nodes())
          if gurobi_qubo_value:
            row.append(np.round(gurobi_qubo_value,4))
            row_d['value_gurobi'] = np.round(gurobi_qubo_value,4)
          else:
            row.append(0)
            row_d['value_gurobi'] = 0
          if gurobi_qubo_tte:
            row.append(np.round(gurobi_qubo_tte,4))
            row_d['gurobi_tte'] = np.round(gurobi_qubo_tte,4)
          else:
            row.append(0)
            row_d['gurobi_tte'] = 0

        report_file_obj = open(os.path.join(report_filename),'a+')
        report_file_obj.write('__'.join(map(str,row))+'\n')
        report_file_obj.close()
        table_contents.append(row)

        with open(report_filename_csv, mode='a', newline='') as report_file:
          writer = csv.DictWriter(report_file, fieldnames=row_d.keys())
          writer.writerow(row_d)

        print('\n')

seed: 222, number of agents: 474
Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-25
gurobi_qubo_value -12155.509999999998
gurobi_qubo_tte: 9.57767939567566
GUROBI QUBO DONE!


seed: 222, number of agents: 475
gurobi_qubo_value -12160.160000000002
gurobi_qubo_tte: 10.56004524230957
GUROBI QUBO DONE!


seed: 222, number of agents: 476
gurobi_qubo_value -12160.159999999998
gurobi_qubo_tte: 7.718837738037109
GUROBI QUBO DONE!


seed: 222, number of agents: 477
gurobi_qubo_value -12160.16
gurobi_qubo_tte: 11.651005029678345
GUROBI QUBO DONE!


seed: 222, number of agents: 478
gurobi_qubo_value -12165.809999999998
gurobi_qubo_tte: 9.951291799545288
GUROBI QUBO DONE!


seed: 222, number of agents: 479
gurobi_qubo_value -12219.81
gurobi_qubo_tte: 5.716696500778198
GUROBI QUBO DONE!


seed: 222, number of agents: 480
gurobi_qubo_value -12219.809999999998
gurobi_qubo_tte: 11.052318811416626
GUROBI QUBO DONE!


seed: 222, number of agents: 481
gurobi_qubo_v